## Section 2: Natural Language Processing (NLP)

In section 1, we explore machine learning classification using traditional as well as deep learning methods utilizing transformer architecture. In this section, we will explore natural language processing (NLP) using the same transformer architecture. We will also explore the use of pre-trained models such as BERT in NLP.

In [1]:
%mkdir models

In [3]:
import pandas as pd
import warnings

# suppress warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# load the processed data
df = pd.read_csv('patients_with_ratings.csv')

# preview the data
display(df.head())

,patient_id,age,gender,medical_history,deterioration_label,timestamp,hear_rate,blood_pressure_sys,blood_pressure_dia,oxygen_saturation,...,has_cancer,has_heart attack,has_heart failure,has_copd,has_asthma,has_alzheimer,has_dementia,fatigue_level,activity_level,mental_health_level
0,9b04b,65,Male,History of hypertension and type 2 diabetes.,True,2023-10-27T10:00:00Z,95.5,160.2,98.7,90.3,...,0,0,0,0,0,0,0,5,1,1
1,bffd5,45,Female,No significant medical history.,False,2023-10-27T10:05:00Z,70.2,120.5,75.0,98.5,...,0,0,0,0,0,0,0,2,4,4
2,fb35e,78,Male,"Chronic obstructive pulmonary disease (COPD), ...",True,2023-10-27T10:10:00Z,105.0,150.0,90.0,88.0,...,0,1,0,1,0,0,0,5,2,1
3,1e30e,30,Female,Mild asthma.,False,2023-10-27T10:15:00Z,65.0,110.0,70.0,99.0,...,0,0,0,0,1,0,0,1,5,4
4,116a4,55,Male,High cholesterol.,False,2023-10-27T10:20:00Z,75.5,135.0,85.0,97.0,...,0,0,0,0,0,0,0,3,3,3


## 1.1 Sentiment Analysis with BERT

In this section, we will explore the use of BERT for sentiment analysis on our dataset. To allow us to perform fine-tuning on BERT, we will make use of

### 1.1.1 Data Preprocessing

We will create classes for the questionnaire data so that it can be used for training and testing. We will train large-BERT models lifestyle.

In [ ]:
# Create sentiment labels for each column
def label_sentiment(df):
    # Copy the dataframe to avoid modifying the original
    df_sentiment = df.copy()

    # Activity level: 1-2 negative, 3 neutral, 4-5 positive (reversed scale)
    df_sentiment['activity_sentiment'] = df_sentiment['activity_level'].apply(
        lambda x: 'negative' if x in [1, 2] else ('neutral' if x == 3 else 'positive')
    )

    return df_sentiment

# Apply the sentiment labeling
df_with_sentiment = label_sentiment(df)

# Display the original values and the new sentiment labels
display(df_with_sentiment[['activity_level', 'activity_sentiment']].head(10))

# Count the sentiment distributions for each category
print("\nActivity Sentiment Distribution:")
print(df_with_sentiment['activity_sentiment'].value_counts())

,fatigue_level,fatigue_sentiment,activity_level,activity_sentiment,mental_health_level,mental_health_sentiment
0,5,negative,1,negative,1,positive
1,2,positive,4,positive,4,negative
2,5,negative,2,negative,1,positive
3,1,positive,5,positive,4,negative
4,3,neutral,3,neutral,3,neutral
5,4,negative,1,negative,1,positive
6,1,positive,5,positive,5,negative
7,5,negative,1,negative,1,positive
8,2,positive,3,neutral,3,neutral
9,4,negative,2,negative,1,positive



Fatigue Sentiment Distribution:
fatigue_sentiment
positive    513
negative    508
neutral     182
Name: count, dtype: int64

Activity Sentiment Distribution:
activity_sentiment
negative    628
positive    378
neutral     197
Name: count, dtype: int64

Mental Health Sentiment Distribution:
mental_health_sentiment
positive    571
negative    460
neutral     172
Name: count, dtype: int64


In [ ]:
# construcut sentiment dataset
df_sentiment = df_with_sentiment[['activity_sentiment']].copy()

# add describe_fatigue, describe_activity, and describe_mental_health columns from the original dataset
df_sentiment['describe_lifestyle'] = df['describe_lifestyle']

# preview the sentiment dataset
display(df_sentiment.head())

,fatigue_sentiment,activity_sentiment,mental_health_sentiment,describe_fatigue_level,describe_lifestyle,describe_mental_health
0,negative,negative,positive,"Severe fatigue, difficulty getting out of bed.","Sedentary, poor diet.",Feeling anxious and low.
1,positive,positive,negative,Mild fatigue occasionally.,"Active, balanced diet.",Generally good.
2,negative,negative,positive,Constant exhaustion.,Very limited activity due to breathlessness.,Feeling down and worried.
3,positive,positive,negative,Not fatigued.,"Very active, regular exercise.",Good.
4,neutral,neutral,neutral,Moderate fatigue after work.,Moderately active.,Stable.


We can see that the data is being properly prepared to be used for training and testing.

### 1.1.2 Model Training

In this section we will train 3 separate BERT models on A100 GPU.

In [10]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [11]:
!nvidia-smi

Fri May 23 11:59:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             43W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# Function to prepare dataset for BERT
def prepare_data_for_bert(text_column, label_column, test_size=0.2):
    # Handle NaN values
    valid_indices = text_column.notna()
    texts = text_column[valid_indices].tolist()
    labels = label_column[valid_indices].tolist()

    # Convert string labels to integers
    label_map = {'positive': 0, 'neutral': 1, 'negative': 2}
    labels = [label_map[label] for label in labels]

    # Split data into train and test sets
    train_texts, test_texts, train_labels, test_labels = train_test_split(
        texts, labels, test_size=test_size, random_state=42, stratify=labels
    )

    return train_texts, test_texts, train_labels, test_labels

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Function to tokenize and encode the texts
def encode_texts(texts, max_length=128):
    return tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

# Function to create torch dataset
def create_dataset(texts, labels):
    encodings = encode_texts(texts)
    input_ids = encodings['input_ids']
    attention_mask = encodings['attention_mask']
    labels = torch.tensor(labels)
    return TensorDataset(input_ids, attention_mask, labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Function to train the model - modified to return evaluation data
def train_bert_model(train_dataset, val_dataset, num_labels=3, epochs=40):
    # Initialize the model
    model = BertForSequenceClassification.from_pretrained(
        'bert-large-uncased',
        num_labels=num_labels
    ).to(device)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)

    # Set up optimizer
    optimizer = AdamW(model.parameters(), lr=2e-5)

    # Track losses
    train_losses = []
    val_losses = []

    # Training loop
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        # Training phase
        model.train()
        train_loss = 0
        train_progress = tqdm(train_loader, desc="Training")

        for batch in train_progress:
            batch = tuple(b.to(device) for b in batch)
            input_ids, attention_mask, labels = batch

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

            train_progress.set_description(f"Training (loss={loss.item():.4f})")

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"Training loss: {avg_train_loss:.4f}")

        # Validation phase for each epoch
        model.eval()
        val_loss = 0
        epoch_preds = []
        epoch_true = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                batch = tuple(b.to(device) for b in batch)
                input_ids, attention_mask, labels = batch

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()

                logits = outputs.logits
                preds = torch.argmax(logits, dim=1)
                epoch_preds.extend(preds.cpu().numpy())
                epoch_true.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Validation loss: {avg_val_loss:.4f}")

    # Final evaluation phase
    model.eval()
    val_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation"):
            batch = tuple(b.to(device) for b in batch)
            input_ids, attention_mask, labels = batch

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    print(f"Validation loss: {avg_val_loss:.4f}")

    # Return the model and evaluation results
    return model, true_labels, predictions, train_losses, val_losses

# Function to run sentiment analysis for each aspect - modified to return evaluation results
def analyze_sentiment(df, text_column, label_column, model_name):
    print(f"\n--- {model_name} Sentiment Analysis Training ---\n")

    # Prepare data
    train_texts, test_texts, train_labels, test_labels = prepare_data_for_bert(
        df[text_column], df[label_column]
    )

    # Create datasets
    train_dataset = create_dataset(train_texts, train_labels)
    test_dataset = create_dataset(test_texts, test_labels)

    # Train model and get evaluation data
    model, true_labels, predictions, train_losses, val_losses = train_bert_model(train_dataset, test_dataset)

    # Save model
    model_path = f"models/bert_{model_name.lower().replace(' ', '_')}"
    model.save_pretrained(model_path)
    print(f"Model saved to {model_path}")

    return model, model_name, true_labels, predictions, train_losses, val_losses

# Dictionary to store evaluation results
eval_results = {}
loss_history = {}

# Run sentiment analysis for each aspect
print("\n=== Training All Sentiment Analysis Models ===\n")

activity_model, activity_name, activity_true, activity_pred, activity_train_loss, activity_val_loss = analyze_sentiment(
    df_sentiment,
    'describe_lifestyle',
    'activity_sentiment',
    'Activity'
)
eval_results['Activity'] = (activity_true, activity_pred)
loss_history['Activity'] = (activity_train_loss, activity_val_loss)


=== Training All Sentiment Analysis Models ===


--- Fatigue Sentiment Analysis Training ---



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/50


...

Training loss: 0.0702
Epoch 50/50


Training (loss=0.0007): 100%|██████████| 61/61 [00:05<00:00, 11.95it/s]


Training loss: 0.0699


Validation: 100%|██████████| 16/16 [00:00<00:00, 38.78it/s]


Validation loss: 0.6435
Model saved to models/bert_mental_health


In [14]:
# Display all evaluation reports after all models are trained
print("\n\n" + "="*70)
print("EVALUATION REPORTS FOR ALL SENTIMENT ANALYSIS MODELS")
print("="*70)

label_names = ['positive', 'neutral', 'negative']
for model_name, (true_labels, predictions) in eval_results.items():
    print(f"\n{model_name} Model Evaluation:")
    print("-" * 40)
    print(classification_report(true_labels, predictions, target_names=label_names))
    print("\n")

import pickle
# Save the evaluation results to a pickle file
with open('models/nlp_eval_results.pkl', 'wb') as f:
    pickle.dump(eval_results, f)



EVALUATION REPORTS FOR ALL SENTIMENT ANALYSIS MODELS

Fatigue Model Evaluation:
----------------------------------------
              precision    recall  f1-score   support

    positive       0.98      0.91      0.94       103
     neutral       0.69      0.77      0.73        35
    negative       0.94      0.97      0.96       102

    accuracy                           0.92       240
   macro avg       0.87      0.88      0.88       240
weighted avg       0.92      0.92      0.92       240




Activity Model Evaluation:
----------------------------------------
              precision    recall  f1-score   support

    positive       0.87      0.88      0.88        76
     neutral       0.72      0.67      0.69        39
    negative       0.94      0.95      0.94       126

    accuracy                           0.88       241
   macro avg       0.84      0.83      0.84       241
weighted avg       0.88      0.88      0.88       241




Mental Health Model Evaluation:
---------

In [ ]:
# Plot training and validation loss for each model with best epoch highlighted
import matplotlib.pyplot as plt
import os
import numpy as np

os.makedirs('images', exist_ok=True)

plt.figure(figsize=(15, 10))

for i, (model_name, (train_losses, val_losses)) in enumerate(loss_history.items(), 1):
    plt.subplot(len(loss_history), 1, i)

    # Plot loss curves
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')

    # Find the best epoch (lowest validation loss)
    best_epoch = np.argmin(val_losses)
    best_val_loss = val_losses[best_epoch]

    # Highlight the best epoch with a star marker
    plt.plot(best_epoch, best_val_loss, 'r*', markersize=12,
             label=f'Best Epoch ({best_epoch+1}): {best_val_loss:.4f}')

    # Add text annotation for clarity - using offset to avoid overlapping
    # Calculate text position to avoid overlapping with the curve
    text_x = best_epoch + len(val_losses) * 0.05  # Offset by 5% of total epochs
    text_y = best_val_loss

    # Make sure text stays within plot boundaries
    text_x = min(text_x, len(val_losses) * 0.7)  # Keep within 70% of plot width

    plt.annotate(f'Best Epoch: {best_epoch+1}\nLoss: {best_val_loss:.4f}',
                xy=(best_epoch, best_val_loss),
                xytext=(text_x, text_y),
                arrowprops=dict(facecolor='black', shrink=0.05, width=1.5))

    plt.title(f'{model_name} Model - Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Print detailed information to verify
    print(f"{model_name} model - Best epoch: {best_epoch+1} with validation loss: {best_val_loss:.4f}")

plt.tight_layout()
plt.savefig('images/sa_loss_curves_with_best_epoch.png')
plt.show()

In [15]:
# zip the models directory
!zip -r models.zip models

# download from colab
from google.colab import files
files.download('models.zip')

  adding: models/ (stored 0%)
  adding: models/bert_activity/ (stored 0%)
  adding: models/bert_activity/model.safetensors (deflated 7%)
  adding: models/bert_activity/config.json (deflated 51%)
  adding: models/bert_fatigue/ (stored 0%)
  adding: models/bert_fatigue/model.safetensors (deflated 7%)
  adding: models/bert_fatigue/config.json (deflated 51%)
  adding: models/nlp_eval_results.pkl (deflated 97%)
  adding: models/bert_mental_health/ (stored 0%)
  adding: models/bert_mental_health/model.safetensors (deflated 7%)
  adding: models/bert_mental_health/config.json (deflated 51%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>